In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

# Census & gmaps API Keys
from config import (api_key)
c = Census(api_key, year=2018)

# Configure gmaps
#gmaps.configure(api_key=gkey)

### The poverty numbers in the dataframe is "Number of persons whose income in the past 12 months is below the poverty level"

In [20]:
#"B25077_001E": "Median Home Value"

census_data = c.acs5.get(("NAME",
                          "B01003_001E",
                          "B17001_002E", 
                          "B17001A_002E", "B17001B_002E", 
                          "B17001D_002E", "B17001I_002E"), 
                         {'for': 'county:*','in':'state:19'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B17001A_002E": "Poverty White",
                                      "B17001B_002E": "Poverty Black", 
                                      "B17001D_002E": "Poverty Asian", 
                                      "B17001I_002E": "Poverty Hispanic",
                                      "NAME": "Name"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Total Poverty Rate (%)"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Name", "Population",
                       "Poverty Count", "Poverty White", "Poverty Black", 
                       "Poverty Asian", "Poverty Hispanic", "Total Poverty Rate (%)"]]


split = census_pd["Name"].str.split(",", n = 1, expand = True)
census_pd["County"]= split[0]
census_pd["State"]=split[1]
census_pd = census_pd[["State", "County", "Population", "Poverty Count", 
                       "Poverty White", "Poverty Black", "Poverty Asian", 
                       "Poverty Hispanic", "Total Poverty Rate (%)"]]

#census_pd.to_csv("2018 Poverty Data", encoding="utf-8", index=False)
census_pd.head()

,State,County,Population,Poverty Count,Poverty White,Poverty Black,Poverty Asian,Poverty Hispanic,Total Poverty Rate (%)
0,Iowa,Adair County,7330.0,687.0,649.0,5.0,4.0,23.0,9.372442
1,Iowa,Adams County,3822.0,368.0,336.0,14.0,0.0,9.0,9.628467
2,Iowa,Allamakee County,14001.0,1388.0,1296.0,23.0,13.0,299.0,9.913578
3,Iowa,Appanoose County,12602.0,1998.0,1947.0,6.0,0.0,37.0,15.854626
4,Iowa,Audubon County,5793.0,651.0,619.0,0.0,7.0,21.0,11.237701
